In [ ]:
pip install webdriver-manager
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import re
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# 함수 정의
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

def remove_tags(element):
    result = ''
    try:
        result = re.sub('<.+?>', '', str(element), 0).strip()
        return result
    except NoSuchElementException:
        return ''


In [ ]:
# 중지 버튼 설정 <- 아직 작동 안 함
stop_button = widgets.Button(description="중지")
display(stop_button)

stop_thread = False

def on_button_clicked(b):
    global stop_thread
    stop_thread = True

stop_button.on_click(on_button_clicked)

# WebDriver 설정
chrome_options = Options()
chrome_service = Service('')
driver = webdriver.Chrome(options=chrome_options, service=chrome_service)

# 검색어 입력
keyword = input("검색어를 입력하세요.")
baseUrl = 'https://www.daangn.com'
url = baseUrl+'/search/'+keyword


driver.get(url)

In [ ]:
html = ''
result_flea = True
try:
    check_exists_by_xpath('//*[@id="flea-market-wrap"]')
    result_flea = True
except:
    result_flea = False
    
if(result_flea == True) :
    count = 0 
    while True:
        if stop_thread:
            print('사용자에 의해 종료되었습니다.')
            break
        try:
            more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@class="more-text"]'))
            )
            more_button.click() 
            time.sleep(1) 
        except:
            html = bs(driver.page_source, 'html.parser')
            break

In [ ]:
html = bs(driver.page_source, 'html.parser')
products = html.select('#flea-market-wrap > article')

prod_list = []

for product in products :
    prod_list.append(product)

datas = []
for prod in prod_list :
    container = dict()
    try:
        container['title'] = remove_tags(prod.find('img')['alt'])
        container['img'] = (prod.find('img')['src'])
        container['content'] = remove_tags(prod.find('span', {'class' : 'article-content'}))
        container['region'] = remove_tags(prod.find('p', {'class' : 'article-region-name'}))
        container['price'] = remove_tags(prod.find('p', {'class' : 'article-price'}))
        container['link'] = baseUrl+remove_tags(prod.find('a', {'class' : 'flea-market-article-link'})['href'])
    except :
        print('error', prod)
    datas.append(container)

# print(datas)

In [ ]:
import pandas as pd

df = pd.DataFrame(datas)
# csv파일로 저장
df.to_csv('./'+keyword+'.csv', index=False)
# 데이터 확인
df